# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np
np.set_printoptions(precision=3)

a3=np.array([[3,15,1,-1,0],[1,5,3,21,-3],[2,10,1,-9,15],[-4,0,12,-5,2],[7,1,31,11,9]])

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [3]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [5]:
a1 = a.copy()
a1[1, 1] = 3
print(a1)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]


Resulting matix still has full rank, but the naive LU routine breaks down.

### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [25]:
def MinorCheck(A):
    N=np.size(A,1)
    checkStatus=True
    breakInd=None
    for ind in range(N):
        if np.abs(np.linalg.det(A[:ind,:ind]))<1e-15:
            checkStatus=False
            breakInd=ind
            break
    return (checkStatus, breakInd)

print(MinorCheck(a3))

(False, 2)


### Вариант 1

Пусть $A\Pi=LU$

Отличие от простейшего алгоритма - перестановки столбцов. Тогда для получения матрицы $U$ на каждом шаге $j$ строим матрицу $U^{(j)}$ путем домножения матрицы $U^{(j-1)}$ ($U^{(0)}$=A) справа на матрицу текущих перестановок $P^{(j)}$ и слева - на матрицу $\Lambda^{(j)}$, получаемую тем же способом, что и раньше:

$U^{(j)} =\Lambda^{(j)} U^{(j-1)} P^{(j)}$

На каждом шаге матрица перестановок $\Pi^{(j)}=\Pi^{(j-1)} P^{(j)}$, $\Pi^0=E$

В результате:

$L^{-1} A \Pi=\Lambda^{(N)}  ... \Lambda^{(1)}  A P^{(1)} ... P^{(N)} $

где

$L=\Lambda^{(N)}  ... \Lambda^{(1)}$

$\Pi = P^{(1)} ... P^{(N)}$

In [27]:
def diy_lup_col(A):
    N=np.size(A,1)
    U = A.copy()
    L = np.eye(N)
    P = np.eye(N)
    
    for j in range(N-1):
        Pj=np.eye(N)
        indMax=j+np.argmax(np.abs(U[j,j:]))
        if indMax!=j:
            Pj[:,[j,indMax]]=Pj[:,[indMax,j]]
            U = U @ Pj
            P = P @ Pj
            
            lam = np.eye(N)
            gamma = U[j+1:, j] / U[j, j]
            lam[j+1:, j] = -gamma
            U = lam @ U

            lam[j+1:, j] = gamma
            L = L @ lam
        
    return L, U, P

### Вариант 2

Пусть $\Pi A=LU$

Отличие от простейшего алгоритма - перестановки строк. Тогда для получения матрицы $U$ на каждом шаге $j$ строим матрицу $U^{(j)}$ путем домножения матрицы $U^{(j-1)}$ ($U^{(0)}$=A) сначала на матрицу текущих перестановок $P^{(j)}$, а потом на матрицу $\Lambda^{(j)}$, получаемую тем же способом, что и раньше:

$U^{(j)} =\Lambda^{(j)} P^{(j)} U^{(j-1)}$

На каждом шаге матрица перестановок $\Pi^{(j)}=P^{(j)} \Pi^{(j-1)}$, $\Pi^0=E$

В результате:

$L^{-1}\Pi A=\Lambda^{(N)} P^{(N)} \Lambda^{(N-1)} P^{(N-1)} ... \Lambda^{(1)} P^{(1)} A$

Отсюда 

$(L^{-1}\Pi-\Lambda^{(N)} P^{(N)} \Lambda^{(N-1)} P^{(N-1)} ... \Lambda^{(1)} P^{(1)}E)A=0$

$L^{-1}\Pi=\Lambda^{(N)} P^{(N)} \Lambda^{(N-1)} P^{(N-1)} ... \Lambda^{(1)} P^{(1)}E$

$\Pi^TL=P^{(1)} \Lambda^{(1)^{-1}}  ...  P^{(N)} \Lambda^{(N)^{-1}} E$

и $L=\Pi P^{(1)} \Lambda^{(1)^{-1}}  ...  P^{(N)} \Lambda^{(N)^{-1}} E$

где $\Pi = P^{(N)}...P^{(1)}$

In [28]:
def diy_lup_row(A):
    N=np.size(A,1)
    U = A.copy()
    L = np.eye(N)
    P = np.eye(N)
    
    for j in range(N-1):
        Pj=np.eye(N)
        indMax=j+np.argmax(np.abs(U[j:,j]))
        if indMax!=j:
            Pj[[j,indMax]]=Pj[[indMax,j]]
            U = Pj @ U
            P = Pj @ P
        
        lam = np.eye(N)
        gamma = U[j+1:, j] / U[j, j]
        lam[j+1:, j] = -gamma
        U = lam @ U

        lam[j+1:, j] = gamma
        L = L @ Pj @ lam
    return P@L, U, P 

In [42]:
lc, uc, pc = diy_lup_col(a3)
lr, ur, pr = diy_lup_row(a3)

In [43]:
print(a3)

[[ 3 15  1 -1  0]
 [ 1  5  3 21 -3]
 [ 2 10  1 -9 15]
 [-4  0 12 -5  2]
 [ 7  1 31 11  9]]


In [44]:
print(lc)

[[ 1.     0.     0.     0.     0.   ]
 [ 0.333  1.     0.     0.     0.   ]
 [ 0.667 -0.391  1.     0.     0.   ]
 [ 0.    -0.234  0.094  1.     0.   ]
 [ 0.067  0.519  0.763  2.281  1.   ]]


In [45]:
print(uc)

[[15.    -1.     0.     1.     3.   ]
 [ 0.    21.333 -3.     2.667  0.   ]
 [ 0.     0.    13.828  1.375  0.   ]
 [ 0.     0.     0.    12.496 -4.   ]
 [ 0.     0.     0.     0.    15.923]]


In [46]:
print(pc)

[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [47]:
print(lc @ uc @ pc.T)

[[ 3. 15.  1. -1.  0.]
 [ 1.  5.  3. 21. -3.]
 [ 2. 10.  1. -9. 15.]
 [-4.  0. 12. -5.  2.]
 [ 7.  1. 31. 11.  9.]]


In [48]:
print(lr)

[[ 1.     0.     0.     0.     0.   ]
 [ 0.429  1.     0.     0.     0.   ]
 [-0.571  0.039  1.     0.     0.   ]
 [ 0.143  0.333  0.088  1.     0.   ]
 [ 0.286  0.667  0.011 -0.394  1.   ]]


In [49]:
print(ur)

[[  7.      1.     31.     11.      9.   ]
 [  0.     14.571 -12.286  -5.714  -3.857]
 [  0.      0.     30.196   1.51    7.294]
 [  0.      0.      0.     21.2    -3.644]
 [  0.      0.      0.      0.     13.484]]


In [50]:
print(pr)

[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [52]:
print(pr.T @ lr @ ur)

[[ 3.000e+00  1.500e+01  1.000e+00 -1.000e+00  0.000e+00]
 [ 1.000e+00  5.000e+00  3.000e+00  2.100e+01 -3.000e+00]
 [ 2.000e+00  1.000e+01  1.000e+00 -9.000e+00  1.500e+01]
 [-4.000e+00  1.384e-17  1.200e+01 -5.000e+00  2.000e+00]
 [ 7.000e+00  1.000e+00  3.100e+01  1.100e+01  9.000e+00]]
